In [ ]:
# %%capture
# !pip install nltk
# import nltk
# nltk.download('words')

In [21]:
# import pandas as pd
import modin.pandas as pd
# import pandas

#############################################
### For the purpose of timing comparisons ###
#############################################
import time
import modin
modin.config.Engine.put("Dask")

from distributed import Client
client = Client()


Perhaps you already have a cluster running?
Hosting the HTTP server on port 40121 instead


In [60]:
import swifter


--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------



In [23]:
dataset_path = "/home/ubuntu/WORK/selfexplain-semantic-topic-modeling/dataset/tripadvisor_raw.csv"
# dataset_path = "/content/1.tripadvisor_df.csv"
# dataset_path = "2_tripadvisor_df_review_1807.csv"

In [49]:
import tqdm as notebook_tqdm

df = pd.read_csv(dataset_path)
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 533430 entries, 0 to 533429
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0.1            533430 non-null  int64  
 1   Unnamed: 0              533430 non-null  int64  
 2   user                    533430 non-null  object 
 3   user_helpful_votes      436162 non-null  object 
 4   user_contribution       462777 non-null  object 
 5   user_rate               533430 non-null  int64  
 6   user_Value              368161 non-null  float64
 7   user_Location           368061 non-null  float64
 8   user_Cleanliness        369361 non-null  float64
 9   user_Service            477812 non-null  float64
 10  review_title            527768 non-null  object 
 11  review_text             482656 non-null  object 
 12  hotel                   533430 non-null  int64  
 13  hotel_name              533351 non-null  object 
 14  hotel_url      

In [50]:
# Drop nan review_text
df = df[df['review_text'].notnull()]
df = df[df['review_title'].notnull()]
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 482649 entries, 0 to 533429
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0.1            482649 non-null  int64  
 1   Unnamed: 0              482649 non-null  int64  
 2   user                    482649 non-null  object 
 3   user_helpful_votes      389823 non-null  object 
 4   user_contribution       415653 non-null  object 
 5   user_rate               482649 non-null  int64  
 6   user_Value              329178 non-null  float64
 7   user_Location           329376 non-null  float64
 8   user_Cleanliness        330420 non-null  float64
 9   user_Service            430624 non-null  float64
 10  review_title            482649 non-null  object 
 11  review_text             482649 non-null  object 
 12  hotel                   482649 non-null  int64  
 13  hotel_name              482578 non-null  object 
 14  hotel_url           

In [51]:
df['hotel'].nunique()

7624

In [52]:
df.sample(5)

,Unnamed: 0.1,Unnamed: 0,user,user_helpful_votes,user_contribution,user_rate,user_Value,user_Location,user_Cleanliness,user_Service,...,hotel_url,hotel_number_of_rating,hotel_number_of_review,hotel_address,hotel_province,hotel_overall_rating,hotel_Value,hotel_Location,hotel_Cleanliness,hotel_Service
451204,493582,7438,maddyc253,2,NaN,5,5.0,NaN,NaN,5.0,...,https://www.tripadvisor.com/Hotel_Review-g2980...,#61 of 382 hotels in Hoi An,469 reviews,"12 Nguyen Du, Cam Pho, Hoi An 560000 Vietnam",QuangNam,5.0,5.0,5.0,5.0,5.0
365039,334645,70437,bartj403,NaN,2,5,3.0,4.0,4.0,4.0,...,https://www.tripadvisor.com/Hotel_Review-g2939...,"#75 of 1,374 hotels in Ho Chi Minh City",459 reviews,"21 - 23 Nguyen Thi Minh Khai Street, District ...",HoChiMinh,4.0,4.0,4.5,4.5,4.5
318514,535339,49195,ThomasJessy,7,10,5,5.0,5.0,5.0,5.0,...,https://www.tripadvisor.com/Hotel_Review-g2980...,#25 of 440 B&Bs / Inns in Hoi An,333 reviews,"484 Hai Ba Trung Street, Hoi An Vietnam",QuangNam,5.0,5.0,4.5,5.0,5.0
495567,280627,16419,senator_traveller,50,95,4,5.0,4.0,5.0,5.0,...,https://www.tripadvisor.com/Hotel_Review-g2939...,"#21 of 1,374 hotels in Ho Chi Minh City","4,488 reviews","235 Nguyen Van Cu St., Ho Chi Minh City Vietnam",HoChiMinh,4.5,4.5,4.0,5.0,4.5
258848,176559,63260,Peter A,8,43,5,5.0,5.0,NaN,5.0,...,https://www.tripadvisor.com/Hotel_Review-g2939...,"#25 of 1,199 hotels in Hanoi","2,077 reviews","21 Hang Khay Street Hoan Kiem District, Hanoi ...",HaNoi,5.0,5.0,5.0,5.0,5.0


In [53]:
df = df.sort_values(by=['hotel'], ascending=[True])

In [54]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'user', 'user_helpful_votes',
       'user_contribution', 'user_rate', 'user_Value', 'user_Location',
       'user_Cleanliness', 'user_Service', 'review_title', 'review_text',
       'hotel', 'hotel_name', 'hotel_url', 'hotel_number_of_rating',
       'hotel_number_of_review', 'hotel_address', 'hotel_province',
       'hotel_overall_rating', 'hotel_Value', 'hotel_Location',
       'hotel_Cleanliness', 'hotel_Service'],
      dtype='object')

In [55]:
df['review'] = df['review_title'] + ". " + df['review_text']
df = df[['hotel', 'review']]

In [56]:
df['hotel'].value_counts().describe()

count    7624.000000
mean       63.306532
std       202.878767
min         1.000000
25%         2.000000
50%         6.000000
75%        32.000000
max      3225.000000
Name: count, dtype: float64

In [57]:
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 482649 entries, 223117 to 520412
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hotel   482649 non-null  int64 
 1   review  482649 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.0+ MB


# Split review by sentences for sentiment analysis

In [79]:
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download('punkt')

docs = ["How are you? I am good", "Good night! Okay!"]
sentences = [sent_tokenize(abstract) for abstract in docs]
sentences = [sentence for doc in sentences for sentence in doc]
sentences

['How are you?', 'I am good', 'Good night!', 'Okay!']

In [89]:
df['hotel'].value_counts().describe()

count    119.000000
mean      60.915966
std       17.601319
min       40.000000
25%       46.000000
50%       55.000000
75%       74.000000
max      108.000000
Name: count, dtype: float64

In [80]:
def split_sent(sents):
    sentences = [sent_tokenize(abstract) for abstract in sents]
    return [sentence for doc in sentences for sentence in doc]

In [94]:
# Group the DataFrame by the 'Category' column
grouped_df = df.groupby('hotel')

sent_df = {
    'hotel': [],
    'sentence': []
}

N = 100000
N_sents = 1000
# Iterate over the groups and perform operations
for group_name, group_data in grouped_df:
    if len(sent_df) > N:
        break
    print(f"Split sentence in {group_name}")
    sens = split_sent(group_data['review'].tolist())
#     print(sens)
    
    
    for i in range(min(len(sens), N_sents)):
        sen = sens[i]
        sent_df['hotel'].append(int(group_name))
        sent_df['sentence'].append(sen)
#     break

Split sentence in 299586
Split sentence in 299783
Split sentence in 300325
Split sentence in 300810
Split sentence in 301984


Split sentence in 302627
Split sentence in 302746
Split sentence in 302750
Split sentence in 302751
Split sentence in 302784
Split sentence in 302836
Split sentence in 302838
Split sentence in 302847


Split sentence in 304523
Split sentence in 306185
Split sentence in 306225
Split sentence in 313715
Split sentence in 451099


Split sentence in 456978
Split sentence in 528397
Split sentence in 530532
Split sentence in 577218
Split sentence in 578971


Split sentence in 582872
Split sentence in 588636
Split sentence in 612806
Split sentence in 614747
Split sentence in 623868


...

Split sentence in 7309526
Split sentence in 7369358
Split sentence in 7617524
Split sentence in 7856034
Split sentence in 7915166


Split sentence in 8595053
Split sentence in 8615276
Split sentence in 8643198
Split sentence in 8773680
Split sentence in 9595723


Split sentence in 9761938
Split sentence in 9786699
Split sentence in 10150919
Split sentence in 10311875
Split sentence in 10432881


Split sentence in 10768488
Split sentence in 11928570
Split sentence in 12158095
Split sentence in 12213991
Split sentence in 12250031


Split sentence in 12275911
Split sentence in 12645634
Split sentence in 15118858


In [95]:
df2 = pd.DataFrame(sent_df)
df2.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 48198 entries, 0 to 48197
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hotel     48198 non-null  int64 
 1   sentence  48198 non-null  object
dtypes: int64(1), object(1)
memory usage: 753.2+ KB


In [100]:
print(df2.head())
print(df2['hotel'].value_counts().describe())

    hotel                                           sentence
0  299586  Why do you ask for a deposit I ' m on business...
1  299586  Upon check in , i was to pay a deposit , since...
2  299586  This is not my first stay at the hotel , and i...
3  299586  , I ' m not sure this policy only to this spec...
4  299586  Maybe you can find other less direct than a de...
count    119.000000
mean     405.025210
std      119.489412
min      227.000000
25%      311.500000
50%      372.000000
75%      489.000000
max      768.000000
Name: count, dtype: float64


In [102]:
df = df2

In [108]:
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 48198 entries, 0 to 48197
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hotel     48198 non-null  int64 
 1   sentence  48198 non-null  object
dtypes: int64(1), object(1)
memory usage: 753.2+ KB


In [104]:
folder = '/home/ubuntu/WORK/selfexplain-semantic-topic-modeling/dataset'
df.to_csv(folder +'/2_tripadvisor_len_review_227per_hotel.csv', index = False)

### Filter hotel have more than N reviews

In [72]:
N = 500
filtered_df = df[(df.groupby('hotel')['hotel'].transform('count') >= N )
                 & (df.groupby('hotel')['hotel'].transform('count') < 2000)]
filtered_df.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 197561 entries, 222313 to 494825
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hotel   197561 non-null  int64 
 1   review  197561 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.5+ MB


In [73]:
filtered_df['hotel'].value_counts().describe()

count     212.000000
mean      931.891509
std       404.206733
min       502.000000
25%       605.750000
50%       783.500000
75%      1156.250000
max      1982.000000
Name: count, dtype: float64

In [74]:
filtered_df.sample(5)

,hotel,review
532218,7379325,"very good . Hotel is very nice , is nice and f..."
489576,302751,Nice stay . We stayed here after out of anothe...
274596,2555547,"Best Hotel in , worth a We stayed for two nigh..."
276120,302627,Solid Choice in . I stayed here with for a few...
289466,11928570,"Great Hotel . Great hotel in District , close ..."


In [17]:
# df = filtered_df
# df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202157 entries, 54831 to 8944
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hotel   202157 non-null  int64 
 1   review  202157 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [75]:
df_ = filtered_df.sample(10000, random_state=29)
df_['hotel'].value_counts().describe()


count    212.000000
mean      47.169811
std       20.698912
min       18.000000
25%       31.000000
50%       43.000000
75%       57.250000
max      108.000000
Name: count, dtype: float64

In [76]:
df_ = df_[(df_.groupby('hotel')['hotel'].transform('count') >= 40)]
df_.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 7249 entries, 5515 to 291943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hotel   7249 non-null   int64 
 1   review  7249 non-null   object
dtypes: int64(1), object(1)
memory usage: 169.9+ KB


In [77]:
df = df_

In [78]:
folder = '/home/ubuntu/WORK/selfexplain-semantic-topic-modeling/dataset'
df.to_csv(folder +'/1_tripadvisor_hotel_review.csv', index = False)

### replace_double_punctuation

In [69]:
import re

def replace_double_punctuation(input_string):
    """
    Replace consecutive occurrences of double punctuation marks with a single punctuation mark.

    Parameters:
        input_string (str): The input string to be processed.

    Returns:
        str: The input string with double punctuation replaced by single punctuation.
    """
    # Define the regular expression pattern to match double punctuation
    pattern = r'(\W|\W\s)\1+'

    # Use re.sub() to replace double punctuation with a single punctuation
    processed_string = re.sub(pattern, r'\1', input_string)
    
#     pattern = r'(\W\s\W)'
#     processed_string = re.sub(pattern, r'', processed_string)

    return processed_string

# Test the function
input_string = "Hellooo!!!! How . . . are you?? I'm - -\. dooing-,. great!!"
output_string = replace_double_punctuation(input_string)
print(output_string)

Hellooo! How . are you? I'm - -\. dooing-,. great!


In [70]:
s = "Hellooo! How are you? I'm\. dooing-,. great!"
# nltk.wordpunct_tokenize(s)

['Hellooo',
 '!',
 'How',
 'are',
 'you',
 '?',
 'I',
 "'",
 'm',
 '\\.',
 'dooing',
 '-,.',
 'great',
 '!']

In [71]:
df['review'] = df['review'].swifter.apply(lambda r: replace_double_punctuation(r))

### Filter English word and keep some character.
Run serveral time to preprocess


In [58]:
import nltk
import string
words = set(nltk.corpus.words.words())
# print(string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

sent = "Io, andiamo to ../ the/ beach? with my a."
# sent = str(df.loc[514675]['review'])
[w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or string.punctuation.find(w)>-1]

['Io', ',', 'to', 'the', '/', 'beach', '?', 'with', 'my', 'a', '.']

In [59]:
def filter_English_words(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(str(sent)) \
        if w.lower() in words or string.punctuation.find(w)>-1)

In [61]:
df['review'] = df['review'].swifter.apply(lambda r: filter_English_words(r))

In [62]:
df = df[df['review'].str.strip().notnull()]
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 482649 entries, 223117 to 520412
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hotel   482649 non-null  int64 
 1   review  482649 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.0+ MB


In [63]:
df['review'].sample(10)

372134    Excellent hotel with hospitable staff I and my...
338462    Great service and very central . My wife and I...
322749    Premier lounge . It was a great experience Spe...
206989    hostel We have been in Brother Inn for nights ...
137022    Great Location and Staff . The location of the...
424876    Good Location . The hotel is near the boat doc...
275316    Modern design hotel . I got days stayed at thi...
75933     Great location , very nice hotel Recently stay...
141482    Fabulous service - very The staff in this hote...
29247     Great Hotel Sunshine hotel is clean and bright...
Name: review, dtype: object

In [64]:
df['review'][:10].str.split()

223117    [THE, place, to, stay, We, stayed, only, night...
405402    [Very, good, ., Came, here, with, &, younger, ...
405436    [Lovely, ., It, ', s, expensive, but, we, coul...
202206    [Amazing, If, you, want, to, have, a, special,...
151660    [The, best, way, to, understand, ., Can, t, im...
405522    [one, of, my, Top, Five, definitely, a, great,...
13705     [Gorgeous, hotel, ., This, is, a, gorgeous, ho...
171564    [Definitely, the, of, ., What, a, lovely, hote...
13714     [Fantastic, ., Top, notch, hotel, ., Great, se...
344976    [Beautiful, ,, historic, *, hotel, in, a, good...
Name: review, dtype: object

In [ ]:
# nltk.wordpunct_tokenize(df.loc[533429].review)

In [ ]:
# df.loc[533429].review.split('.')

### split_by_multiple_chars

using split sentence for SELFEXPLAN in later step

In [ ]:
import re

#               !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
def split_by_multiple_chars(input_string):
    """
    Split a string by multiple characters: ';', ':', and ','.

    Parameters:
        input_string (str): The input string to be split.

    Returns:
        list: A list of substrings after splitting.
    """
    # Define the regular expression pattern with multiple characters as delimiters
    pattern = r'[.?!\s]+'  # This will split by ';', ':', ',', or any whitespace characters.

    # Use re.split() to split the string by the defined pattern
    substrings = re.split(pattern, input_string)

    return substrings

# Test the function
input_string = "apple;orange,banana:grape apple?? orange, banana: grape"
output = split_by_multiple_chars(input_string)
print(output)


#### Remove NAME in review


In [65]:
temp_df = df[df['review'].str.contains("# NAME ?")]['review']
print(temp_df.info())
temp_df

<class 'modin.pandas.series.Series'>
Index: 15 entries, 137792 to 471025
Series name: review
Non-Null Count  Dtype 
--------------  ----- 
15 non-null     object
dtypes: object(1)
memory usage: 240.0+ bytes
None


137792            Great place with a good location # NAME ?
407212    # NAME Location is VERY GOOD The room I had in...
417960    # NAME - Well old building on the river front ...
274344    # NAME A better price / quality will be hard t...
414403    # NAME Fantastic and lovely hotel . Warm welco...
351548                   A good resort with nice . # NAME ?
343795         Excellent value and great service . # NAME ?
261228               Nice Hotel in a good Location # NAME ?
417961    # NAME - great little hotel with exceptional s...
206464    # NAME After an hectic bus trip from Ho Chi Mi...
489299                              Great location # NAME ?
66004     # NAME I took the night bus from and at . The ...
148949                                      Nice . # NAME ?
329492    # NAME + Location of Hotel Da really convenien...
471025                               Wonder stay . # NAME ?
Name: review, dtype: object

In [66]:
# Remove NAME word in review - bad collecting
df['review'] = df['review'].apply(lambda r: r.replace("# NAME ?", ""))

#### Remove nan in review

In [67]:
df[df['review']=='nan']

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


,hotel,review


In [68]:
df = df[df['review'] != 'nan']
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 482649 entries, 223117 to 520412
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hotel   482649 non-null  int64 
 1   review  482649 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.0+ MB


#### Remove to short text

In [ ]:
MIN_REVIEW_LENGTH = 50
temp_df = df[df['review'].str.len() < MIN_REVIEW_LENGTH]
print(temp_df.info())
temp_df.sample(10)

In [ ]:
# df = df[df['review'].str.len() >= MIN_REVIEW_LENGTH]
# df.info()

In [ ]:
df['review'].sample(10)

## Get label by rating

In [ ]:
df.info()

In [ ]:
def get_label(rate):
    return 1 if rate > 3.5 else 0

In [ ]:
df['label'] = df['hotel_overall_rating'].apply(lambda r: get_label(r))
df.sample(10)

In [109]:
# df['label'].value_counts()

### Get label by textbob

In [ ]:
%%capture
!pip install textblob

In [ ]:
from textblob import TextBlob


In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    """
    Get the sentiment (positive or negative) of the input text.

    Parameters:
        text (str): The input text.

    Returns:
        str: The sentiment, either 'positive' or 'negative'.
    """
    # Create a TextBlob object to analyze the sentiment
    blob = TextBlob(text)

    # Get the sentiment polarity score (-1 to +1)
    sentiment_score = blob.sentiment.polarity

    # Determine the sentiment based on the polarity score
    if sentiment_score >= 0:
        return 1 # 'positive'
    # elif sentiment_score < 0:
    else:
        return 0 #'negative'
    # else:
    #     return 'neutral'

# Test the function
text1 = "I love this product. It's fantastic!"
text2 = "I'm really disappointed with the service."

print("Sentiment for text1:", get_sentiment(text1))
print("Sentiment for text2:", get_sentiment(text2))

In [ ]:
df['label_textblob'] = df['review'].apply(lambda r: get_sentiment(r))
df.sample(10)

In [ ]:
df['label_textblob'].value_counts()

In [ ]:
df[df['label_textblob']==0].sample(10)

### Get sentiment by Vander

In [105]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

def get_sentiment_vader(text):
    """
    Get the sentiment (positive, neutral, or negative) of the input text using VADER.

    Parameters:
        text (str): The input text.

    Returns:
        str: The sentiment, either 'positive', 'neutral', or 'negative'.
    """
    # Create a SentimentIntensityAnalyzer object
    sia = SentimentIntensityAnalyzer()

    # Get the polarity scores for the input text
    sentiment_scores = sia.polarity_scores(text)

    # Determine the sentiment based on the compound score
    compound_score = sentiment_scores['compound']
    if compound_score >= 0.05:
        return 1 #'positive'
    # elif compound_score < -0.05:
    else:
        return 0 # 'negative'
    # else:
    #     return 'neutral'

# Test the function
text1 = "I love this product. It's fantastic!"
text2 = "I'm really disappointed with the service."

print("Sentiment for text1:", get_sentiment_vader(text1))
print("Sentiment for text2:", get_sentiment_vader(text2))

Sentiment for text1: 1
Sentiment for text2: 0


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [113]:
import swifter

df['label_vader'] = df['sentence'].swifter.apply(lambda r: get_sentiment_vader(r))
df.sample(10)

,hotel,sentence,label_vader
7226,306225,"If the hotel resume the door person , I ' d li...",1
48034,15118858,A warm welcome to from the O ' Gallery Majesti...,1
25408,2340470,"The was a stay with and payment , kindly they ...",1
17663,1090179,We thoroughly our stay .,0
44368,9786699,Stay with these you wont regret it,1
12841,623868,"The staff are friendly and helpful , especiall...",1
37887,6533098,The staff - .,0
33378,4456285,"Stayed there recently , it was nice , room mod...",1
27322,2557683,The were absolutely delicious and to all .,1
39551,7182682,That ' s when you can walk without the being m...,0


In [119]:
df['label_vader'].value_counts()

label_vader
1    34035
0    14163
Name: count, dtype: int64

In [115]:
df[df['label_vader']==0].sample(10)

,hotel,sentence,label_vader
42785,8643198,It was within a short walking distance to the ...,0
26582,2521380,Pool and is a break for the hustle and bustle ...,0
41215,7856034,You made our visit to and our anniversary even...,0
3000,302746,Majestic .,0
26520,2521380,We spent four nights and were .,0
47886,12645634,This place is incredible .,0
27703,2557683,The were very clean but we did change because ...,0
18982,1420780,She went further to help us and our problem .,0
6274,302847,", We had a luxury room and could see the sea !",0
12982,623907,You feel a bit removed from real life and the ...,0


In [121]:
df.to_csv(folder +'/3_tripadvisor_len_review_227per_hotel_label_vader.csv', index = False)

# 

In [120]:
# from sklearn.model_selection import train_test_split

# df3 = pd.DataFrame({
#     "sentence": df['sentence'].tolist(),
#     "label": df['label_vader'].tolist()
# })

# print(df3.info())

# # Assuming you have your feature data in X and label data in y

# # Split the data into training and testing sets, stratifying based on the label y
# X_train, X_test, y_train, y_test = train_test_split(df3, df3, test_size=0.2, stratify=df3['label'], random_state=29)

# # Now, you have X_train, X_test, y_train, y_test with preserved label distribution.
# print(X_train.info())

# print("----\n")

# print(y_test.info())

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 48198 entries, 0 to 48197
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  48198 non-null  object
 1   label     48198 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 753.2+ KB
None
<class 'modin.pandas.dataframe.DataFrame'>
Index: 38558 entries, 43737 to 18691
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  38558 non-null  object
 1   label     38558 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 903.7+ KB
None
----

<class 'modin.pandas.dataframe.DataFrame'>
Index: 9640 entries, 8037 to 31218
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  9640 non-null   object
 1   label     9640 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 225.9+ KB
None


# Group by label and hotel

In [ ]:
grouped_count = df.groupby(['hotel', 'label_vader']).size().reset_index(name='Count')
grouped_count.sort_values(by=['Count'], ascending=[True])

In [ ]:
df2 = grouped_count
df2[df2['label_vader']==1]

# Select some records/Rows

In [ ]:
negative_idx = df[df['label_vader']==0].index
negative_idx

In [ ]:
df.loc[negative_idx]

In [ ]:
# positive
df1 = df[df['hotel'].isin(df.loc[negative_idx]['hotel'].tolist())].sample(32000, random_state=29)

In [ ]:
# negative
df2 = df.loc[negative_idx]

In [ ]:
df3 = pd.concat([df1,df2], ignore_index = True)
df3.info()

In [ ]:
df3

In [ ]:
df3['label_vader'].value_counts()

In [ ]:
df3.to_csv("/home/ubuntu/WORK/selfexplain-semantic-topic-modeling/SelfExplain/data/RoBERTa-Tripadvisor"
           + '/2_tripadvisor_labels_sub.csv', index=False)

#### Check different value

## Sort by hotel_id

In [ ]:
df = df.sort_values(by=['hotel'], ascending=[True])

In [ ]:
df.head()

In [ ]:
df.info()

## Create dataset for SELFEXPLAIN

In [ ]:
# Save processed dataset
folder = "/home/ubuntu/WORK/selfexplain-semantic-topic-modeling/dataset"
df.to_csv(folder + '/2_tripadvisor_labels.csv', index=False)

In [ ]:
folder = "/home/ubuntu/WORK/selfexplain-semantic-topic-modeling/SelfExplain/data/RoBERTa-Tripadvisor/"

In [ ]:
sub_df = df3
# sub_df = df

In [ ]:
pd.DataFrame({
    "sentence": sub_df['review'].tolist(),
    "label": sub_df['label_textblob'].tolist()
}).to_csv(folder + 'dataset_textblob.tsv', sep='\t', index=False)

In [ ]:
pd.DataFrame({
    "sentence": sub_df['review'].tolist(),
    "label": sub_df['label_vader'].tolist()
}).to_csv(folder + 'dataset_vander.tsv', sep='\t', index=False)